In [2]:
pip install lpips torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lpips

Note: you may need to restart the kernel to use updated packages.


In [5]:
import lpips

In [22]:
# Src (psnr): copied from https://www.geeksforgeeks.org/python-peak-signal-to-noise-ratio-psnr/
# Src (ssim): copied from https://medium.com/@danielyogatama.dy/ssim-on-python-eb1a76a2799b

from math import log10, sqrt 
import cv2 
import numpy as np 
import os
from os import listdir
import cv2
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim

import lpips
import torch
from PIL import Image
import torchvision.transforms as transforms

# https://github.com/richzhang/PerceptualSimilarity/tree/master?tab=readme-ov-file#a-basic-usage
def lpips_compare(img1_path, img2_path):
    # Read the image (https://www.tutorialspoint.com/how-to-normalize-an-image-in-opencv-python)
    image = cv2.imread(img1_path)
    image2 = cv2.imread(img2_path)

    # Normalize the image
    img_normalized = cv2.normalize(image, None, -1.0, 1.0,
    cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    img_normalized2 = cv2.normalize(image2, None, -1.0, 1.0,
    cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    # Define a transform to convert
    # the image to torch tensor
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    # Convert the image to Torch tensor
    tensor = transform(img_normalized)
    tensor2 = transform(img_normalized2)
    
    loss_fn = lpips.LPIPS(net='alex')
    d = loss_fn.forward(tensor,tensor2)
    
    return d.item()
    

def ssim_compare(img1_path, img2_path) :    
    img1 = cv2.imread(img1_path, 0)
    img2 = cv2.imread(img2_path, 0)
    ssim_score, dif = ssim(img1, img2, full=True)    
    return ssim_score

# for each pair of imagesets compute average psnr, ssim, lpips
def process_folders(ref_dir, in_dir):
    ref_files = [f for f in os.listdir(ref_dir) if f.endswith(".jpg")]
    in_files = [f for f in os.listdir(in_dir) if f.endswith(".jpg")]

    psnr_values = []
    ssim_values = []
    lpips_values = []
    
    for ref_file, in_file in zip(ref_files, in_files):
        original_path = os.path.join(ref_dir, ref_file)
        compressed_path = os.path.join(in_dir, in_file)

        original = cv2.imread(original_path)
        compressed = cv2.imread(compressed_path)

        if original.shape != compressed.shape:
            print(f"Warning: Dimensions mismatch for {ref_file} and {in_file}")
            continue

        psnr_value = psnr(original, compressed)
        psnr_values.append(psnr_value)

        ssim_value = ssim_compare(original_path, compressed_path)
        ssim_values.append(ssim_value)

        lpips_value = lpips_compare(original_path, compressed_path)
        lpips_values.append(lpips_value)
        
        print(f"PSNR for {ref_file} and {in_file}: {psnr_value:.2f} dB")
        print(f"SSIM for {ref_file} and {in_file}: {ssim_value:.3f}")
        print(f"LPIPS for {ref_file} and {in_file}: {lpips_value:.3f}")
    
    return psnr_values, ssim_values, lpips_values

def psnr(original, compressed): 
    mse = np.mean((original - compressed) ** 2) 
    if(mse == 0):  # MSE is zero means no noise is present in the signal . 
                  # Therefore PSNR have no importance. 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr 
  
def main(): 
    # declare your reference and input (comparing) folder
    ref_dir = "reference_images"
    in_dir = "input_images"

    psnr_values, ssim_values, lpips_values = process_folders(ref_dir, in_dir)

    # average psnr values
    if psnr_values:
        avg_psnr = sum(psnr_values) / len(psnr_values)
        print(f"\nAverage PSNR: {avg_psnr:.2f} dB")
    else:
        print("\nNo valid PSNR calculations performed.")

    # average ssim values
    if psnr_values:
        avg_ssim = sum(ssim_values) / len(ssim_values)
        print(f"Average SSIM: {avg_ssim:.3f}")
    else:
        print("\nNo valid SSIM calculations performed.")

    # average lpips values
    if lpips_values:
        avg_lpips = sum(lpips_values) / len(lpips_values)
        print(f"Average LPIPS: {avg_lpips:.3f}")
    else:
        print("\nNo valid LPIPS calculations performed.")

if __name__ == "__main__": 
    main()

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: C:\Users\Other\anaconda3\Lib\site-packages\lpips\weights\v0.1\alex.pth
PSNR for image_1.jpg and image_1.jpg: 32.34 dB
SSIM for image_1.jpg and image_1.jpg: 0.804
LPIPS for image_1.jpg and image_1.jpg: 0.207
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: C:\Users\Other\anaconda3\Lib\site-packages\lpips\weights\v0.1\alex.pth
PSNR for image_16.jpg and image_16.jpg: 29.17 dB
SSIM for image_16.jpg and image_16.jpg: 0.546
LPIPS for image_16.jpg and image_16.jpg: 0.510
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: C:\Users\Other\anaconda3\Lib\site-packages\lpips\weights\v0.1\alex.pth
PSNR for image_23.jpg and image_23.jpg: 28.05 dB
SSIM for image_23.jpg and image_23.jpg: 0.547
LPIPS for image_23.jpg and image_23.jpg: 0.540
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: